Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [371]:
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse
from functools import reduce
from copy import copy

In [372]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [373]:
NUM_POINTS = 10

In [374]:
x = make_set_covering_problem(NUM_POINTS, NUM_POINTS, .3)
#print("Element at row=42 and column=42:", x[42, 42])

In [375]:
sets = x.toarray()
sets

array([[False,  True,  True, False, False,  True, False, False, False,
         True],
       [False, False,  True, False, False, False, False,  True, False,
        False],
       [False,  True, False,  True, False, False,  True, False,  True,
         True],
       [ True, False, False, False, False, False, False,  True,  True,
        False],
       [False, False, False,  True, False,  True,  True, False,  True,
        False],
       [False, False,  True, False, False, False, False, False, False,
         True],
       [False, False,  True,  True,  True, False, False, False,  True,
        False],
       [False,  True,  True, False, False, False, False, False,  True,
         True],
       [ True, False, False, False, False, False,  True,  True, False,
         True],
       [ True, False,  True, False,  True, False,  True,  True, False,
        False]])

In [376]:
from collections import namedtuple
State = namedtuple('State', ['taken', 'not_taken'])


In [377]:
def fitness(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [sets[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost

In [378]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, NUM_POINTS - 1)
    #print(index, new_state[index])
    new_state[index] = not new_state[index]
    return new_state

In [379]:
current_state = [choice([False]) for _ in range(NUM_POINTS)]
print(current_state)

print(fitness(current_state))

for step in range(1_000):
    new_state = tweak(current_state)
    if fitness(new_state) >= fitness(current_state):
        current_state = new_state
        #print(step)
        print(fitness(current_state))

[False, False, False, False, False, False, False, False, False, False]
(0, 0)
0
(3, -1)
1
(7, -2)
2
(9, -3)
3
(10, -4)


In [380]:
current_state

[True, False, False, True, True, False, True, False, False, False]